## 10. Advanced Topics
### 10.1 Circular References

Description
* in some languages, such as C# or Java, circular references are easy
* but in Rust, they are complicated because of the borrow checker & object lifetime(s) 
* a circular reference occures when two functions call on each other
  * if there is a bad implementation (i.e. no termination condition), then the code could run in a infinite loop

Example
* imagine a class-scheduling programm: where each student can take multiple courses and a course can have multiple students
  * a user will want to 1) call a student to see what courses they are enrolled in and 2) call a course to see which students are attending it
  * there relationship is n:n
  * `struct Student` and `struct Course` will need to have the same lifetime 

<p align="center"><img src="circular_reference_example.png" ></p>

Solution 
* however, circular referencing and borrowing of lifetimes is not recommended (as native references are static)
* using the reference counter `std::rc::Rc` and the reference cell `std:cell::RefCell`, we can implement this
* the [RefCell](https://doc.rust-lang.org/std/cell/index.html#refcellt) provides "dynamic borrowing" 
* immutable references can be called from `RefCell` with `.borrow()` and mutuable ones with `-borrow_mut()`
*


```Rust
// circular implementation with generic lifetimes
// this is not going to work in Rust

struct Student<'a>{
    name: String,
    courses: Vec<&'a Course<'a>>
}

impl<'a> Student<'a>{
    fn new(name: &str) -> Student<'a>{
        Student(name: name.into(), courses: Vec::new())}
}

struct Course<'a>{
    name: String,
    students: Vec<&'a Student<'a>>
}

impl<'a> Course<'a>{
    fn new(name: &str) -> Course<'a>{
        Course(name: name.into(), students: Vec::new())}
    fn add_student(&'a mut self, student:&'a mut Student<'a>){
        student.courses.push(self);
        self.students.push(student);
    }
}
```

In [2]:
use std::rc::Rc;
use std::cell::RefCell;

struct Student {
    name: String,
    courses: Vec<Rc<RefCell<Course>>>
}

impl Student {
    fn new(name: &str) -> Student{
        Student{name: name.into(), courses: Vec::new(),}
    }
}
 
struct Course {
    name: String,
    students: Vec<Rc<RefCell<Student>>>
}

impl Course {
    fn new(name: &str) -> Course{
        Course{name: name.into(), students: Vec::new(),}
    }

    fn add_student(course: Rc<RefCell<Course>>, student: Rc<RefCell<Student>>){
        student.borrow_mut().courses.push(course.clone());
        course.borrow_mut().students.push(student);
    }
}

In [3]:
fn circular_demo(){
    let john = Rc::new(RefCell::new(Student::new("John")));

    let math = Course::new("Math");
    let magic_course = Rc::new(RefCell::new(math));

    Course::add_student(magic_course, john);
}

circular_demo()

()